# Red-Black BST

- This notebook represents an ordered symbol table of generic key-value pairs. It supports the usual <em>put</em>, <em>get</em>, <em>contains</em>, <em>delete</em>, <em>size</em>, and <em>is-empty</em> methods.

- This implementation uses a <em>left-leaning red-black BST</em>. 

### Big(O)

- The <em>put</em>, <em>get</em>, <em>contains</em>, <em>remove</em>, <em>minimum</em>, <em>maximum</em>, <em>ceiling</em>, <em>floor</em>, <em>rank</em>, and <em>select</em> operations each take &Theta;(log <em>n</em>) time in the worst case, where <em>n</em> is the number of key-value pairs in the symbol table.

- The <em>size</em>, and <em>is-empty</em> operations take &Theta;(1) time.

- The <em>keys</em> methods take <em>O</em>(log <em>n</em> + <em>m</em>) time, where <em>m</em> is the number of keys returned by the iterator. 

- Construction takes &Theta;(1) time.

### Implementation

In [1]:
class Node:
    def __init__(self, key: int, value: str, size: int, color: str):
        self.key = key
        self.value = value
        self.size = size
        self.left = None
        self.right = None
        self.color = color
        
    def __repr__(self):
        lines = []
        if self.right:
            found = False
            for line in repr(self.right).split("\n"):
                if line[0] != " ":
                    found = True
                    line = " ┌─" + str("'"+line+"'")
                elif found:
                    line = " | " + line
                else:
                    line = "   " + line
                lines.append(line)
        lines.append(str(self.value))
        if self.left:
            found = False
            for line in repr(self.left).split("\n"):
                if line[0] != " ":
                    found = True
                    line = " └─" + str("'"+line+"'")
                elif found:
                    line = "   " + line
                else:
                    line = " | " + line
                lines.append(line)
        return "\n".join(lines)
        

In [2]:
class RedBlackBST:
    def __init__(self):
        self.root = None
        
    def size(self, node_x):
        if node_x == None:
            return 0
        else:
            return node_x.size
        
        
    def isEmpty(self):
        return self.size(self.root) == 0
    
        
    def isRed(self, node_x):
        if node_x == None:
            return False
        return node_x.color == 'RED'
    
    def rotate_left(self, node):
        x = node.right
        node.right = x.left
        x.left = node
        x.color = node.color
        node.color = 'RED'
        return x
    
    def rotate_right(self, node):
        x = node.left
        node.left = x.right
        x.right = node
        x.color = node.color
        node.color = 'RED'
        return x
    
    def flip_colors(self, node):
        node.color = 'RED'
        node.left.color = 'BLACK'
        node.right.color = 'BLACK'
    
    def recursive_put(self, node_x, key, value):
        #print('Called')
        if node_x == None:
            #print('Added\n')
            return Node(key, value, 1, 'RED')
        
        if key < node_x.key:
            node_x.left = self.recursive_put(node_x.left, key, value)
        elif key > node_x.key:
            node_x.right = self.recursive_put(node_x.right, key, value)
        else:
            node_x.value = value
            
        if self.isRed(node_x.right) and not self.isRed(node_x.left):
            #print('In here1')
            node_x = self.rotate_left(node_x)
        if self.isRed(node_x.left) and self.isRed(node_x.left.left):
            #print('In here2')
            node_x = self.rotate_right(node_x)
        if self.isRed(node_x.right) and self.isRed(node_x.left):
            #print('In here3')
            self.flip_colors(node_x)
            
        node_x.size = 1 + self.size(node_x.left) + self.size(node_x.right)
        return node_x
    
    def put(self, key, value):
        self.root = self.recursive_put(self.root, key, value)
        self.root.color = 'BLACK'
        
        
    def contains(self, key):
        return self.get(key) != None
    
    
    def get(self, key):
        x = self.root
        while(x != None):
            if key < x.key:
                x = x.left
            elif key > x.key:
                x = x.right
            else:
                return x.value
        
        return None
    
    
    def inorder_traversal(self, node_x, list_keys):
        if node_x == None:
            return
        self.inorder_traversal(node_x.left, list_keys)
        custom_str = 'key:' + str(node_x.key) + '  size:' + str(node_x.size)
        list_keys.append(custom_str)
        self.inorder_traversal(node_x.right, list_keys)
        
        
        
    def level_order_traversal(self, queue, results):
        while(len(queue) > 0):
            results.append(queue[0].key)
            
            node = queue[0]
            queue.pop(0)
            
            
            if node.left is not None:
                queue.append(node.left)
            
            if node.right is not None:
                queue.append(node.right)
        return results
    
    
    def print_tree(self):
        if self.root == None:
            return "Empty Tree\n\n"
        
        return repr(self.root)
    
    
    
    def print_bfs(self):
        results = []
        queue = []
        queue.append(self.root)
        results = self.level_order_traversal(queue, results)
        return str(results)
    
    
    
    def print_dfs(self):
        list_keys = []
        self.inorder_traversal(self.root, list_keys)
        return "\n".join(list_keys)

    
    def rank(self, key):
        return self.recursive_rank(key, self.root)
    
    def recursive_rank(self, key, node_x):
        if node_x == None:
            return 0
        
        if key < node_x.key:
            return self.recursive_rank(key, node_x.left)
        elif key > node_x.key:
            return 1 + self.size(node_x.left) + self.recursive_rank(key, node_x.right)
        else:
            return self.size(node_x.left)
        
    
    
    def range_count(self, lo_key, hi_key):
        if self.contains(hi_key):
            return self.rank(hi_key) - self.rank(lo_key) + 1
        else:
            return self.rank(hi_key) - self.rank(lo_key)
    
        
        

In [3]:
list_x = [26, -68, -66, -29, -89, -67, -8, 64, 71, 19]
rbBST = RedBlackBST()

In [4]:
for x in list_x:
    rbBST.put(x, str(x))

In [5]:
print(rbBST.print_dfs())

key:-89  size:1
key:-68  size:3
key:-67  size:1
key:-66  size:10
key:-29  size:1
key:-8  size:4
key:19  size:1
key:26  size:2
key:64  size:6
key:71  size:1


In [6]:
print(rbBST.print_tree())

    ┌─'71'
 ┌─'64'
 |  |  ┌─'26'
 |  |  |  └─'19'
 |  └─'-8'
 |     └─'-29'
-66
 |  ┌─'-67'
 └─'-68'
    └─'-89'


In [7]:
rbBST.print_bfs()

'[-66, -68, 64, -89, -67, -8, 71, -29, 26, 19]'